In [1]:
## Import dependencies
import pandas as pd
from linearmodels import PanelOLS


In [13]:
data = pd.read_csv("project_data_rates.csv", index_col=0).rename(
    columns={
        "permanent_housing": "permanet_housing_rate",
        "homeless": "homeless_rate",
        "poverty": "poverty_rate",
        "Black or African American": "african_american",
        "American Indian and Alaska Native": "american_indian_n_alaska_native",
        "Asian": "asian",
        "Native Hawaiian and Other Pacific Islander": "pacific_islander",
        "Two or More Races": "multiple_races",
        "Non Hispanic": "non_hispanic",
        "Drug Deaths": "drug_death_rate",
        "Suicide Deaths": "suicide_death_rate",
    }
)
data.head()


,CoC Number,Coc,Year,permanet_housing_rate,homeless_rate,poverty_rate,unemployment_rate,Population,Male,Female,White,african_american,american_indian_n_alaska_native,asian,pacific_islander,multiple_races,non_hispanic,Hispanic,drug_death_rate,suicide_death_rate
0,AZ-502,"Phoenix,Mesa/Maricopa",2007,52.001543,218.127624,98946.775104,3200.0,3872962,50412.475000,49587.525000,87858.904890,4886.079440,2248.950545,3007.620524,231.967161,1766.477440,69301.067245,30698.932755,14.846518,14.665778
1,AZ-502,"Phoenix,Mesa/Maricopa",2008,63.740080,181.620069,98615.857511,5100.0,3958263,50450.159578,49549.840422,87561.084243,5048.527599,2244.696727,3094.817095,236.568414,1814.305922,68666.685362,31333.314638,13.288657,13.642348
2,AZ-502,"Phoenix,Mesa/Maricopa",2009,62.861472,196.091006,98737.277325,8900.0,4023132,50471.871169,49528.128831,87320.152558,5202.364725,2218.719147,3163.182317,240.484279,1855.096974,68167.934833,31832.065167,14.068641,13.024678
3,AZ-502,"Phoenix,Mesa/Maricopa",2010,78.933918,183.358278,98824.767488,9500.0,3817117,49473.594862,50526.405138,85623.940791,5389.722138,2610.949573,3675.155883,264.990567,2435.241047,70429.489062,29570.510938,15.954449,14.775549
4,AZ-502,"Phoenix,Mesa/Maricopa",2011,82.933995,150.463014,98699.221313,8500.0,3875371,49431.628611,50568.371389,85242.470979,5479.423776,2627.387159,3847.347777,266.142261,2537.228049,70307.797628,29692.202372,16.179096,16.024272


In [17]:
mod = PanelOLS.from_formula(
    "homeless_rate ~ permanet_housing_rate + poverty_rate + unemployment_rate + drug_death_rate + suicide_death_rate" 
    "+ White + african_american + american_indian_n_alaska_native + asian + pacific_islander + multiple_races" 
    "+ EntityEffects + TimeEffects",
    data=data.set_index(["CoC Number", "Year"]),
    drop_absorbed=True,
).fit()
mod.summary


Dep. Variable:,homeless_rate,R-squared:,0.2017
Estimator:,PanelOLS,R-squared (Between):,-11.273
No. Observations:,285,R-squared (Within):,-2.2727
Date:,"Sat, Apr 22 2023",R-squared (Overall):,-9.6466
Time:,12:05:10,Log-likelihood,-1700.2
Cov. Estimator:,Unadjusted,,
,,F-statistic:,6.1152
Entities:,19,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(10,242)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,11.443


In [18]:
mod2 = PanelOLS.from_formula(
    "homeless_rate ~ permanet_housing_rate + poverty_rate + unemployment_rate + drug_death_rate + suicide_death_rate" 
    "+ Hispanic + non_hispanic" 
    "+ EntityEffects + TimeEffects",
    data=data.set_index(["CoC Number", "Year"]),
    drop_absorbed=True,
).fit()
mod2.summary


Dep. Variable:,homeless_rate,R-squared:,0.0415
Estimator:,PanelOLS,R-squared (Between):,0.2620
No. Observations:,285,R-squared (Within):,0.1183
Date:,"Sat, Apr 22 2023",R-squared (Overall):,0.2360
Time:,12:07:04,Log-likelihood,-1726.3
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.7743
Entities:,19,P-value,0.1049
Avg Obs:,15.000,Distribution:,"F(6,246)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,9.2920


In [19]:
mod3 = PanelOLS.from_formula(
    "homeless_rate ~ permanet_housing_rate + poverty_rate + unemployment_rate + drug_death_rate + suicide_death_rate" 
    "+ Male + Female" 
    "+ EntityEffects + TimeEffects",
    data=data.set_index(["CoC Number", "Year"]),
    drop_absorbed=True,
).fit()
mod3.summary

Dep. Variable:,homeless_rate,R-squared:,0.0903
Estimator:,PanelOLS,R-squared (Between):,-0.8517
No. Observations:,285,R-squared (Within):,0.1156
Date:,"Sat, Apr 22 2023",R-squared (Overall):,-0.6770
Time:,12:07:52,Log-likelihood,-1718.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.0682
Entities:,19,P-value,0.0007
Avg Obs:,15.000,Distribution:,"F(6,246)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,11.989
